<a href="https://colab.research.google.com/github/KOMBOU12/Marius/blob/main/Fairness_et_donn%C3%A9es_des%C3%A9quilibr%C3%A9es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algorihme 1: random over-sampling**

In [12]:
pip install imbalanced-learn

In [15]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

# Chargement du jeu de données SMS Spam Collection
data = pd.read_csv('SMSSpamCollection.tsv', delimiter='\t', header=None, names=['Label', 'Message'])

# Prétraitement : conversion des labels textuels en numérique
data['Label'] = data['Label'].map({'ham': 0, 'spam': 1})

# Extraction des caractéristiques (X) et des étiquettes (y)
X = data['Message']
y = data['Label']

# Vectorisation des messages texte en utilisant TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_vectorized = vectorizer.fit_transform(X)

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.3, random_state=42)

# Entraînement d'un modèle RandomForest sur les données originales
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Évaluation du modèle sur les données non rééquilibrées
y_pred = model.predict(X_test)
accuracy_original = accuracy_score(y_test, y_pred)
report_original = classification_report(y_test, y_pred)

print("Distribution originale des classes:", Counter(y_train))
print("Précision du modèle sans rééquilibrage:", accuracy_original)
print("Rapport de classification sans rééquilibrage:\n", report_original)

# Application de RandomOverSampler pour rééquilibrer les classes
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Entraînement du modèle sur les données rééquilibrées
model.fit(X_train_resampled, y_train_resampled)

# Évaluation du modèle sur les données rééquilibrées
y_pred_resampled = model.predict(X_test)
accuracy_resampled = accuracy_score(y_test, y_pred_resampled)
report_resampled = classification_report(y_test, y_pred_resampled)

print("Distribution après rééquilibrage:", Counter(y_train_resampled))
print("Précision du modèle avec rééquilibrage:", accuracy_resampled)
print("Rapport de classification avec rééquilibrage:\n", report_resampled)


Distribution originale des classes: Counter({0: 3359, 1: 538})
Précision du modèle sans rééquilibrage: 0.9796529024536206
Rapport de classification sans rééquilibrage:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1463
           1       1.00      0.84      0.91       208

    accuracy                           0.98      1671
   macro avg       0.99      0.92      0.95      1671
weighted avg       0.98      0.98      0.98      1671

Distribution après rééquilibrage: Counter({0: 3359, 1: 3359})
Précision du modèle avec rééquilibrage: 0.9838420107719928
Rapport de classification avec rééquilibrage:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99      1463
           1       0.99      0.88      0.93       208

    accuracy                           0.98      1671
   macro avg       0.99      0.94      0.96      1671
weighted avg       0.98      0.98      0.98      1671



# **Algorihme 2: random under-sampling**

In [11]:
# Importation des bibliothèques nécessaires
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

# Chargement du jeu de données de détection de fraude par carte de crédit

data = pd.read_csv('creditcard.csv')
data = data.dropna()

# Séparation des caractéristiques (X) et des étiquettes (y)
X = data.drop('Class', axis=1)
y = data['Class']

# Division des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardisation des données pour améliorer les performances de la régression logistique
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Entraînement du modèle avant sous-échantillonnage
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Prédiction et évaluation du modèle avant sous-échantillonnage
y_pred = model.predict(X_test)
print("Rapport de classification avant sous-échantillonnage :")
print(classification_report(y_test, y_pred))
print(f"Précision : {accuracy_score(y_test, y_pred) * 100:.2f}%")

# Application du RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

# Entraînement du modèle après sous-échantillonnage
model.fit(X_train_res, y_train_res)

# Prédiction et évaluation du modèle après sous-échantillonnage
y_pred_res = model.predict(X_test)
print("Rapport de classification après sous-échantillonnage :")
print(classification_report(y_test, y_pred_res))
print(f"Précision : {accuracy_score(y_test, y_pred_res) * 100:.2f}%")

Rapport de classification avant sous-échantillonnage :
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     16616
         1.0       0.74      0.57      0.65        40

    accuracy                           1.00     16656
   macro avg       0.87      0.79      0.82     16656
weighted avg       1.00      1.00      1.00     16656

Précision : 99.85%
Rapport de classification après sous-échantillonnage :
              precision    recall  f1-score   support

         0.0       1.00      0.97      0.98     16616
         1.0       0.06      0.93      0.12        40

    accuracy                           0.97     16656
   macro avg       0.53      0.95      0.55     16656
weighted avg       1.00      0.97      0.98     16656

Précision : 96.69%
